In [ ]:
import os
import numpy as np
import pandas as pd
import pandasql as ps
from tqdm import tqdm 
tqdm.pandas(desc="progress-bar")

import gensim 
import string

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

LabeledSentence = gensim.models.doc2vec.LabeledSentence

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
os.chdir('C:\\Users\\BTHANISH\\Documents\\Thanish\\Competition\\Hacker earth\\Amazon Hiring Challenge')


In [2]:
#import tensorflow as tf
#import tensorflow_hub as hub


In [3]:
from bert_embedding import BertEmbedding

In [4]:
DF_train_prod = pd.read_csv('train.csv')
DF_test_prod = pd.read_csv('test.csv')
#DF_test_prod = DF_test_prod.loc[~DF_test_prod.duplicated(),]

print(DF_train_prod.shape, DF_test_prod.shape)

DF_test_prod['topic'] = 'Test_dataset'
DF_prod = pd.concat([DF_train_prod, DF_test_prod]).reset_index(drop = True)
DF_prod.columns = ['Review_text', 'Review_Title', 'topic']
DF_prod.head()

(5959, 3) (2553, 2)


,Review_text,Review_Title,topic
0,"Did nothing for me, didn't help lost even with...",Useless,Shipment and delivery
1,"Did nothing for me, didn't help lost even with...",Useless,Not Effective
2,I have bought these bags and immediately open...,TRASH!!! Do not buy these bags it’s a waist of...,Customer Service
3,Gave me an allergic reaction on my face :(,Do not recommend,Allergic
4,These don't compare to the name brand wipes. F...,Can't tackle big messes,Texture


In [5]:
stop = stopwords.words('english')

# Tokenizing the two columns Review_text, Review_Title
for col in ['Review_text', 'Review_Title']:
    #Converting the columns to lowercase
    DF_prod[col] = DF_prod[col].apply(lambda x :  x.lower())

    #Replacing the puncutations
    DF_prod[col] = DF_prod[col].str.replace('[^\w\s]','')
    
    # Removing stop words
    DF_prod[col] = DF_prod[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    # Remove numerics from the data
    DF_prod[col] = DF_prod[col].str.replace('\d+', '')
    
    #Getting the tokens for the two columns
    new_col = col + '_tokens'
    DF_prod[new_col] = DF_prod.apply(lambda row : word_tokenize(row[col]), axis = 1)

# Combined tokens
DF_prod['combined_tokens'] = DF_prod['Review_text_tokens'] + DF_prod['Review_Title_tokens']
DF_prod.head(10)

,Review_text,Review_Title,topic,Review_text_tokens,Review_Title_tokens,combined_tokens
0,nothing didnt help lost even working eating he...,useless,Shipment and delivery,"[nothing, didnt, help, lost, even, working, ea...",[useless],"[nothing, didnt, help, lost, even, working, ea..."
1,nothing didnt help lost even working eating he...,useless,Not Effective,"[nothing, didnt, help, lost, even, working, ea...",[useless],"[nothing, didnt, help, lost, even, working, ea..."
2,bought bags immediately open one put trash bag...,trash buy bags waist time,Customer Service,"[bought, bags, immediately, open, one, put, tr...","[trash, buy, bags, waist, time]","[bought, bags, immediately, open, one, put, tr..."
3,gave allergic reaction face,recommend,Allergic,"[gave, allergic, reaction, face]",[recommend],"[gave, allergic, reaction, face, recommend]"
4,dont compare name brand wipes family little k...,cant tackle big messes,Texture,"[dont, compare, name, brand, wipes, family, li...","[cant, tackle, big, messes]","[dont, compare, name, brand, wipes, family, li..."
5,dont compare name brand wipes family little k...,cant tackle big messes,Quality/Contaminated,"[dont, compare, name, brand, wipes, family, li...","[cant, tackle, big, messes]","[dont, compare, name, brand, wipes, family, li..."
6,dont compare name brand wipes family little k...,cant tackle big messes,Color and texture,"[dont, compare, name, brand, wipes, family, li...","[cant, tackle, big, messes]","[dont, compare, name, brand, wipes, family, li..."
7,good,tastes horrible,Bad Taste/Flavor,[good],"[tastes, horrible]","[good, tastes, horrible]"
8,extremely hard swallow pills huge sides sharp ...,choking hazard,Too big to swallow,"[extremely, hard, swallow, pills, huge, sides,...","[choking, hazard]","[extremely, hard, swallow, pills, huge, sides,..."
9,first style leaving review solimo incontinent ...,bring back old style,Quality/Contaminated,"[first, style, leaving, review, solimo, incont...","[bring, back, old, style]","[first, style, leaving, review, solimo, incont..."


# Splitting the data back to train prod and test prod

In [60]:
train_prod = DF_prod.loc[DF_prod.topic != 'Test_dataset',].reset_index(drop = True)
test_prod = DF_prod.loc[DF_prod.topic == 'Test_dataset',].reset_index(drop = True)

print(train_prod.shape, test_prod.shape)

# Get the indep and dep features
indep = ['combined_tokens']
dep = ['topic']

(5959, 6) (2553, 6)


In [61]:
# Convert the labels to integer
LE = LabelEncoder()
LE.fit(train_prod.topic)
train_prod.topic = LE.transform(train_prod.topic)


# Splitting to local train and test

In [62]:
train_stop = int(train_prod.shape[0]*0.7)

np.random.seed(100)
# train_local_X, test_local_X, train_local_Y, test_local_Y = train_test_split(train_prod[indep],
#                                                                             train_prod[dep],
#                                                                             test_size = 0.2)
train_local_X = train_prod.loc[0 : train_stop, indep]
train_local_Y = train_prod.loc[0 : train_stop, dep]
test_local_X = train_prod.loc[train_stop : train_prod.shape[0], indep]
test_local_Y = train_prod.loc[train_stop : train_prod.shape[0], dep]

train_prod_X = train_prod[indep]
train_prod_Y = train_prod[dep]
test_prod_X = test_prod[indep]

print(train_local_X.shape, train_local_Y.shape, test_local_X.shape, test_local_Y.shape, train_prod_X.shape, test_prod_X.shape)

(4172, 1) (4172, 1) (1788, 1) (1788, 1) (5959, 1) (2553, 1)


# Bert

In [63]:
# Download the Bert model for the first time using proxy
#os.environ["https_proxy"] = "http://19.12.1.40:83"

#bert_embedding = BertEmbedding(model='bert_24_1024_16', dataset_name='book_corpus_wiki_en_cased')


In [448]:
#Get the word vectors for the sentence
# trian_prod_bert = bert_embedding(train_prod.Review_text)
# test_prod_bert = bert_embedding(test_prod.Review_text)

In [486]:
#Get the sentence level embedding by taking the mean of each words in the test
train_bert_vectors = ([np.mean(trian_prod_bert[i][1], axis = 0) for i in range(len(trian_prod_bert))])
train_bert_vectors[5560] = np.array([0.0] *1024) #since it's a null text in that index replacing it with 0

test_bert_vectors = ([np.mean(test_prod_bert[i][1], axis = 0) for i in range(len(test_prod_bert))])
train_bert_vectors[0:3], test_bert_vectors[0:3]

C:\Users\BTHANISH\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
C:\Users\BTHANISH\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars


([array([-0.04575083,  0.26852962, -0.40489498, ...,  0.11674985,
         -0.02588899,  0.16120338], dtype=float32),
  array([-0.04575083,  0.26852962, -0.40489498, ...,  0.11674985,
         -0.02588899,  0.16120338], dtype=float32),
  array([ 0.33884838, -0.06202189, -0.49974066, ..., -0.3031003 ,
         -0.02780935, -0.16894451], dtype=float32)],
 [array([-0.11446831,  0.14398797, -0.210073  , ...,  0.24956672,
          0.1559298 , -0.02459837], dtype=float32),
  array([-0.11446831,  0.14398797, -0.210073  , ...,  0.24956672,
          0.1559298 , -0.02459837], dtype=float32),
  array([-0.15249276, -0.47609544, -0.75302124, ...,  0.01009943,
         -0.24685921,  0.5926263 ], dtype=float32)])

In [477]:
# for i in range(np.array(test_prod_bert).shape[0]):
#     print(np.mean(test_prod_bert[i][1], axis = 0))

In [478]:
# n_dim = 1024

# #from sklearn.preprocessing import scale
# print('Train prod happening')
# train_prod_vecs_w2v = np.concatenate([buildWordVector(row[indep], bert_length = 100) for i, row in train_prod_X.iterrows()])
# #train_vecs_w2v = scale(train_vecs_w2v)

# print('Test prod happening')
# test_prod_vecs_w2v = np.concatenate([buildWordVector(row[indep], bert_length = 100) for i, row in test_prod_X.iterrows()])
# #test_vecs_w2v = scale(test_vecs_w2v)

# print('Train local happening')
# train_local_vecs_w2v = np.concatenate([buildWordVector(row[indep], bert_length = 100) for i, row in train_local_X.iterrows()])
# #train_vecs_w2v = scale(train_vecs_w2v)

# print('Test local happening')
# test_local_vecs_w2v = np.concatenate([buildWordVector(row[indep], bert_length = 100) for i, row in test_local_X.iterrows()])
# #test_vecs_w2v = scale(test_vecs_w2v)


In [492]:
train_stop = int(train_prod.shape[0]*0.7)

np.random.seed(100)
# train_local_X, test_local_X, train_local_Y, test_local_Y = train_test_split(train_prod[indep],
#                                                                             train_prod[dep],
#                                                                             test_size = 0.2)
train_local_X = np.array(train_bert_vectors[0 : train_stop])
train_local_Y = train_prod.topic[0 : train_stop]

test_local_X = np.array(train_bert_vectors[train_stop : train_prod.shape[0]])
test_local_Y = train_prod.topic[train_stop : train_prod.shape[0]]

train_prod_X = np.array(train_bert_vectors)
train_prod_Y = train_prod.topic
test_prod_X = np.array(test_bert_vectors)

print(train_local_X.shape, train_local_Y.shape, 
      test_local_X.shape, test_local_Y.shape, train_prod_X.shape, test_prod_X.shape
     )

(4171, 1024) (4171,) (1788, 1024) (1788,) (5959, 1024) (2553,)


# XGBOOST

In [423]:
dtrain_local = xgb.DMatrix(data = train_local_X, label = train_local_Y)
dtest_local = xgb.DMatrix(data = test_local_X, label = test_local_Y)
#dtrain_prod = xgb.DMatrix(data = train_prod_X, label = train_prod_Y)
#dtest_prod = xgb.DMatrix(data = test_prod_X)


num_rounds = 1000

params = {'objective' : 'multi:softprob',
          'num_class' : len(train_local_Y.unique()),
          #'eval_metric': 'auc',
          'max_depth' : 7,
          'eta' : 0.3,
          'subsample': 1,
          'colsample_bytree': 1,
          'silent' : 1
          ,'tree_method' : 'gpu_hist'
          }

eval_set = [(dtrain_local,'train'), (dtest_local,'test')]

In [ ]:
#Stopping. Best iteration:
#[106]	train-merror:0.288111	test-merror:0.488255
        
np.random.seed(100)
num_rounds = 1000
np.random.seed(100)
xgb_model = xgb.train(params,
                      dtrain_local,
                      evals = eval_set,
                      num_boost_round = num_rounds,
                      #feval = custom_mse,
                      verbose_eval = True,
                      early_stopping_rounds = 30)

xgb_local_pred = xgb_model.predict(dtest_local)

#Feature importance
pd.DataFrame.from_dict(xgb_model.get_score(), orient = 'index').rename(columns = {0:'importance'}).sort_values(['importance'], ascending = False)
